### 1. Import thư viện cần thiết

In [37]:
import cv2 
import numpy as np

### 2. Đọc và chuyển sang ảnh xám 

In [38]:
img = cv2.imread('D:/python_project/PCA_For_Face_Recognition/test.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Chuyển đổi ảnh sang thang độ xám

### 3. Tạo bộ ảnh phát hiện khuôn mặt (Haar cascade)

In [39]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

### 4. Phát hiện khuôn mặt trong ảnh 

In [40]:
faces = face_cascade.detectMultiScale(gray, 
                                      scaleFactor=1.2, # Tỷ lệ thu phóng trong khoảng 1.1-1.3, ảnh sẽ giảm 20% mỗi lần, giúp phát hiện khuôn mặt nhỏ hơn nhưng chậm
                                      minNeighbors=6, # Số lượng vùng lân cận cần thiết để xác định một khuôn mặt, giá trị từ 3-6, giá trị cao hơn sẽ giảm số lượng phát hiện nhưng tăng độ chính xác
                                      minSize=(20, 20)) # Kích thước tối thiểu của khuôn mặt, giá trị này có thể điều chỉnh tùy theo kích thước khuôn mặt trong ảnh

# Kết quả trả về danh sách các tọa độ (x, y, w, h) của các khuôn mặt phát hiện được

### 5. Tiền xử lý từng khuôn mặt (chuẩn bị cho PCA)

In [41]:
common_size = (100, 100) # Chuẩn hóa kích thước khuôn mặt
face_images = [] # Danh sách lưu trữ các ảnh khuôn mặt đã chuẩn hóa
threshold_energy = 1e6 # Ngưỡng năng lượng để lọc ra những vùng không phải mặt

### 6. Xử lý từng khuôn mặt phát hiện được 

In [42]:
for (x, y, w, h) in faces:
    face = img[y:y+h, x:x+w] # Cắt khuôn mặt từ ảnh gốc
    face_resized = cv2.resize(face, common_size) # Chuẩn hóa kích thước khuôn mặt
    face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY) # Chuyển đổi khuôn mặt sang thang độ xám
    face_vector = face_gray.flatten().astype(np.float32) # Chuyển đổi khuôn mặt thành vector

    # Chuẩn hóa
    face_mean = np.mean(face_vector) # Tính giá trị trung bình của vector khuôn mặt
    face_normalized = face_vector - face_mean # Chuẩn hóa vector khuôn mặt bằng cách trừ đi giá trị trung bình
    energy = np.linalg.norm(face_normalized) ** 2 # Tính năng lượng của khuôn mặt
    face_images.append((face_normalized, energy, (x, y, w, h))) # Lưu trữ khuôn mặt đã chuẩn hóa, năng lượng và tọa độ của khuôn mặt trong danh sách

### 7. Đếm khuôn mặt có năng lượng lớn hơn ngưỡng 

In [43]:
num_detected_faces = sum(energy > threshold_energy for (_, energy, _) in face_images) # Chỉ đếm những vùng khuôn mặt có năng lượng đủ lớn → loại bỏ mặt giả, mặt mờ, ảnh nhiễu.
print('Number of faces recognized:', num_detected_faces)

Number of faces recognized: 8


### 8. Hiển thị khuôn mặt được nhận diện

In [44]:
for (_, energy, (x, y, w, h)) in face_images:
    if energy > threshold_energy:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, 'Face', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

### 9. Hiển thị ảnh kết quả

In [45]:
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()